# Configuration

This section show principles of building configs in nginx.

Here we create the `experiment_nginx` container that we would use for the experiments on this page.

In [1]:
docker run -itd --name experiment_nginx --rm -p 80:80 nginx

5dc76eabfe237931f1bc35c1dde22476c401fb580406b6ba86da497bad6f83a0


**Note** Don't forget to stop the container afte all.

In [1]:
docker stop experiment_nginx

experiment_nginx


## Validate

You can use `nginx -t` syntax to validate the `nginx` config.

---

Suppose we created an nginx config, but made a typo in the word `server`. Let's apply `nginx -t` command in such case:

In [2]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}

http {
    surver { # typo!
        listen 80;
        return 200 "this is new message from nginx";
    }
}
EOF

docker exec experiment_nginx nginx -t

2024/09/04 13:05:19 [emerg] 68#68: unknown directive "surver" in /etc/nginx/nginx.conf:4
nginx: [emerg] unknown directive "surver" in /etc/nginx/nginx.conf:4
nginx: configuration file /etc/nginx/nginx.conf test failed


: 1

We got the message `configuration file /etc/nginx/nginx.conf test failed` and something like tracings.

The following cell fixes the error and re-runs `nginx -t`.

In [3]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}

http {
    server {
        listen 80;
        return 200 "this is new message from nginx";
    }
}
EOF

docker exec experiment_nginx nginx -t

nginx: the configuration file /etc/nginx/nginx.conf syntax is ok
nginx: configuration file /etc/nginx/nginx.conf test is successful


Now we have `syntax ok` - which means that everything is working fine.

### Arbitrary file

You can validate an arbitrary Nginx configuration file using the `-c <file path>` option.

---

The following cell creates an Nginx configuration with a mistake—it's trying to use the `wrong_directive` directive. It saves this configuration to a random path, `/etc/nginx/toy.conf`, but not in the path suitable for Nginx.

In [6]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/toy.conf' <<EOF
events {
    wrong_directive "hello";
}

http {
    server {
        listen 80;
        return 200 "this is new message from nginx";
    }
}
EOF

docker exec experiment_nginx nginx -t -c /etc/nginx/toy.conf

2024/09/04 13:08:01 [emerg] 115#115: unknown directive "wrong_directive" in /etc/nginx/toy.conf:2
nginx: [emerg] unknown directive "wrong_directive" in /etc/nginx/toy.conf:2
nginx: configuration file /etc/nginx/toy.conf test failed


: 1

After validating the file, we received a message indicating that it uses an unknown directive—just as expected.

## Include

You can include other configuration files by using the `include <path to the file>` directive.

---

The following example creates a server that will have two endpoints, `/` and `/temp`. The temp endpoint is placed in the other file, and it is included in the main `server` context using the `include <path to file>` directive. 

In [12]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}

http {
    server {
        listen 80;
        location / {
            return 200 "MAIN configuration";
        }
        include /etc/nginx/temp.conf;
    }
}
EOF

docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/temp.conf' <<EOF
location /temp {
    return 200 "TEMP configuration";
}
EOF

docker exec experiment_nginx nginx -s reload

2024/07/22 09:17:39 [notice] 246#246: signal process started


Now that config has been applied we can try to request it from endpoints.

In [13]:
curl localhost:80

MAIN configuration


In [14]:
curl localhost:80/temp

TEMP configuration


A request to the `/` endpoint returns one message, and `/temp` returns another.